In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.utils import to_categorical,plot_model
#from keras.utils import to_categorical,plot_model
from keras.callbacks import CSVLogger
import pylab as pl
import os
from skimage.transform import resize,rotate
import numpy as np
import pdb

In [4]:
def makemodel():
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(100, 100, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(16, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adadelta')
    return model

In [5]:
##  load full cement img
rootpath = '../input/cementphoto/yes'
imgdir = os.listdir(rootpath)
clsInfo = []
data = []
label = []
ims = []
rescaleSize = (100,100)

for imname in os.listdir(rootpath):
            im0 = pl.imread(os.path.join(rootpath,imname))
            im = resize(im0,rescaleSize,mode='reflect')
            data.append(np.resize(im,(1,)+rescaleSize+(3,)))
            label.append(1)
            ims.append(im0)
            for arg in np.arange(-45,45,10):
                im = resize(rotate(im0,arg,resize=True,cval=1),rescaleSize,mode='reflect')
                data.append(np.resize(im,(1,)+rescaleSize+(3,)))
                label.append(1)
          


id = np.arange(len(label))
print(id)



In [19]:
## load Counterfeits cement img
rootpath = '../input/cementphoto/no'
imgdir = os.listdir(rootpath)
rescaleSize = (100,100)
i = 0
for imname in os.listdir(rootpath):
            im0 = pl.imread(os.path.join(rootpath,imname))
            im = resize(im0,rescaleSize,mode='reflect')
            data.append(np.resize(im,(1,)+rescaleSize+(3,)))
            label.append(0)
            ims.append(im0)
            for arg in np.arange(-45,45,10):
                im = resize(rotate(im0,arg,resize=True,cval=1),rescaleSize,mode='reflect')
                data.append(np.resize(im,(1,)+rescaleSize+(3,)))
                label.append(0)
                i = i + 1
            if i > 50:
                break

In [43]:
## split train data and test data
id = np.arange(len(label))
np.random.shuffle(id)

num = int(0.8*len(label))
trainData = [data[ii] for ii in id[0:num]]
trainLabel = [label[ii] for ii in id[0:num]]
testData = [data[ii] for ii in id[num:]]
#print(testData)
testLabel = [label[ii] for ii in id[num:]]
print(testLabel)

trainData=np.concatenate(trainData,axis=0)
testData = np.concatenate(testData,axis=0)
print(testData[2])
trainY = to_categorical(trainLabel,2)
testY = to_categorical(testLabel,2)



In [25]:

model = makemodel()
plot_model(model,to_file='model_part2.png',show_shapes=True)
csv_logger = CSVLogger('training_part2.log')
model.fit(trainData,trainY,batch_size=64,epochs=50,verbose=1,validation_data=(testData,testY),callbacks=[csv_logger])
predictY = model.predict(testData)
print(predictY)
acc = np.sum(np.array(testLabel) == np.argmax(predictY,axis=1))/float(len(predictY))
print('accurate : %f%%'%(acc*100))

In [59]:
print(testData.shape)
print(predictY.shape)

In [110]:
# Importing all necessary libraries
#decomposing a video into set of images
import cv2 
import os 

cam = cv2.VideoCapture("../input/cement/v2.MOV")
arr=[]
currentframe = 0
while(True): 

    ret,frame = cam.read() 

    if ret: 
         
        if currentframe % 40 == 0:
            name = 'frame' + str(currentframe) + '.jpg'
            arr.append(name)
            print ('Creating...' + name)
            cv2.imwrite(name, frame) 
        currentframe += 1
#         if currentframe % 1000000 ==0:
#             img=cv2.imread(name)
#             height,width,layers = img.shape
#             s = (width,height)
#             arr.append(img)
    else: 
        break


In [109]:
for name in arr:
    os.remove(f"/kaggle/working/{name}")
# os.remove("/kaggle/working/frame100.jpg")
# os.remove("/kaggle/working/frame200.jpg")
# os.remove("/kaggle/working/frame300.jpg")
# os.remove("/kaggle/working/frame400.jpg")
# os.remove("/kaggle/working/frame500.jpg")
# os.remove("/kaggle/working/frame0.jpg")

In [111]:
## load video cement img
rootpath = '/kaggle/working/'
imgdir = os.listdir(rootpath)
clsInfo1 = []
data1 = []
ims1 = []
rescaleSize = (100,100)
i = 0
for imname in arr:
        im0 = pl.imread(os.path.join(rootpath,imname))
        #print(f"im0 =",im0)
        im = resize(im0,rescaleSize,mode='reflect')
        #print(f"im =",im)
        data1.append(np.resize(im,(1,)+rescaleSize+(3,)))
        #print(f"data1 =",data1)
        #i = i + 1
    #label.append(1)
#     ims1.append(im0)
#     for arg in np.arange(-45,45,10):
#         im = resize(rotate(im0,arg,resize=True,cval=1),rescaleSize,mode='reflect')
#         data1.append(np.resize(im,(1,)+rescaleSize+(3,)))
#         #label.append(0)

                




In [112]:
testData1 = data1
#print(testData1)
testData1 = np.concatenate(testData1,axis=0)
#print(testData1.shape)
#print(testData1[0])

In [113]:
predictY1 = model.predict(testData1)
print(predictY1.shape)
finalarr = []
t = 0
for i,j in predictY1:
    print(max(i,j))
    if max(i,j)*100 >= 95:
        finalarr.append(arr[t])
    t = t + 1

#print(finalarr)
#     predictY = model.predict(name)
#     print(predictY)

In [114]:
print(finalarr)

In [115]:
import pytesseract
import numpy as nd
from pytesseract import Output
import cv2
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,30)
from PIL import ImageEnhance, Image

In [116]:
# Generic Libraries
from PIL import Image
import os
import pandas as pd
import numpy as np
import re,string,unicodedata

import pytesseract

#Warnings
import warnings
warnings.filterwarnings("ignore")

#Garbage Collection
import gc

#Gensim Library for Text Processing
import gensim.parsing.preprocessing as gsp
from gensim import utils

#TextBlob Library (Sentiment Analysis)
from textblob import TextBlob, Word

#Plotting Libraries
import matplotlib.pyplot as plt
import seaborn as sns

#WordCloud Generator
from wordcloud import WordCloud,STOPWORDS

In [118]:
time1 = []
detected = []

for name in finalarr:
    print(f"For {name} : ")
    time1.append(name)
    img = cv2.imread(f'/kaggle/working/{name}', -1)
#     img = Image.fromarray(img)
#     enhancer = ImageEnhance.Contrast(img)

#     factor = 1.5 #increase contrast
#     im_output = enhancer.enhance(factor)
#     im_output.save('more-contrast-image.png')

    text = pytesseract.image_to_string(f'/kaggle/working/{name}')
    print(f"text = ",text)
    detected.append(text)
    print("--------------------------------")




   
# dictionary of lists 
dict = {'Frame': time1, 'Text on Bag': detected} 
     
df = pd.DataFrame(dict)
  
print(df)
# saving the dataframe
df.to_csv('file1.csv')